## Install dependencies

In [3]:
!pip install pydub

In [3]:
!pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=9567403331bc537e3ac53769bf6a3a9ae6097b931c28e715fd174a5cc0d85a82
  Stored in directory: /home/ec2-user/.cache/pip/wheels/aa/0d/3c/5a9cbae53044e993a211358ddae115ce34bcbc29f7a3bd9155
Successfully built ffmpeg


### Import dependencies

In [12]:
from pydub import AudioSegment
import os 
import math, random
from IPython.display import Audio

### Helper functions 
* mono_to_stereo converts mono audio files to stereo by replicating the single channel to both the left and reight 
* slice_audio creates 3 second audio segments with rolling window 500 ms 
* process_audio is a wrapper function for the above 2 functions

In [13]:
def mono_to_stereo(file_path):
    
    left_channel = AudioSegment.from_wav(file_path)
    right_channel = AudioSegment.from_wav(file_path)

    stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
    file_no_ext = os.path.splitext(file_path)[0]
    stereo_sound.export(file_no_ext+'_stereo.wav', format="wav")
    return file_no_ext+'_stereo.wav' 

def slice_audio(filename, category):
    base = os.path.basename(filename)
    input_file = os.path.splitext(base)[0]
    extension = os.path.splitext(base)[1]
    audio = AudioSegment.from_file(filename)
    length = audio.duration_seconds
    if not os.path.isdir('Training_audio/'+category): 
        os.mkdir('Training_audio/'+category)
    for i in range(0, (int(length)-3)*1000, 500):
        start = i
        end = i+3000
        filename1 = input_file+'_'+str(start)+'_'+str(end)+extension
        #print(input_file)
        #print(filename1)
        newAudio = AudioSegment.from_wav(filename)
        newAudio = newAudio[start:end]
        newAudio.export('Training_audio/'+category+'/'+filename1, format="wav")


In [14]:
def process_audio(file_path, category):
    sound = AudioSegment.from_file(file_path)
    channel_count = sound.channels
    if channel_count == 1: 
        file_path = mono_to_stereo(file_path)
    slice_audio(file_path, category)

### Process all audio files into 3 second clips

In [29]:
import os
input_dir = 'Audio/curlew'

for subdir, dirs, files in os.walk(input_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        process_audio(file_path, 'curlew')
        

In [30]:
import os
input_dir = 'Audio/whipbird'

for subdir, dirs, files in os.walk(input_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        process_audio(file_path, 'whipbird')


In [30]:
import os
input_dir = 'Audio/background'

for subdir, dirs, files in os.walk(input_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        process_audio(file_path, 'background')


### Manual Processing
Listen to each 3 second clip
If it contains at least 1 full distinctive example of this bird's call, keep it
If it only contains a partial example of this bird's call, discard it
If it contains another bird's call, discard it, even if it also contains the bird you are listening for

Keep approved files in the existing directory

#### Note: If you wish to listen to the audio segments within this notebook, below is an example

In [42]:
from IPython.display import Audio
Audio('Training_audio/curlew/eastern_curlew_long_1_10000_13000.wav')